In [1]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


### Loading the data

In [2]:

# Define constants
train_dir = 'Potato/train/'
test_dir = 'Potato/test/'
batch_size = 32
num_classes = 3
input_shape = (250, 250, 3)  # Adjust according to your image dimensions

# Function to filter images based on extensions
def filter_images_by_extension(directory):
    valid_extensions = ['.jpg', '.jpeg', '.png']
    image_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if any(file.lower().endswith(ext) for ext in valid_extensions):
                image_files.append(os.path.join(root, file))
    return image_files

# Filter image files
train_image_files = filter_images_by_extension(train_dir)
test_image_files = filter_images_by_extension(test_dir)


In [3]:

# Create DataFrame for training and testing
train_df = pd.DataFrame({
    'filename': train_image_files,
    'class': [os.path.basename(os.path.dirname(path)) for path in train_image_files]
})

test_df = pd.DataFrame({
    'filename': test_image_files,
    'class': [os.path.basename(os.path.dirname(path)) for path in test_image_files]
})

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Normalization for testing
test_datagen = ImageDataGenerator(rescale=1./255)

# Generating batches of training and testing data
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='class',
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical')


Found 5702 validated image filenames belonging to 3 classes.
Found 1426 validated image filenames belonging to 3 classes.


### Training the model

In [4]:

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)



Epoch 1/10
179/179 [==============================] - 138s 766ms/step - loss: 0.5499 - accuracy: 0.7799 - val_loss: 0.1910 - val_accuracy: 0.9264
Epoch 2/10
179/179 [==============================] - 160s 892ms/step - loss: 0.1674 - accuracy: 0.9372 - val_loss: 0.2143 - val_accuracy: 0.9257
Epoch 3/10
179/179 [==============================] - 172s 957ms/step - loss: 0.1003 - accuracy: 0.9640 - val_loss: 0.2737 - val_accuracy: 0.9067
Epoch 4/10
179/179 [==============================] - 177s 985ms/step - loss: 0.0777 - accuracy: 0.9712 - val_loss: 0.0964 - val_accuracy: 0.9656
Epoch 5/10
179/179 [==============================] - 169s 941ms/step - loss: 0.0694 - accuracy: 0.9740 - val_loss: 0.3095 - val_accuracy: 0.9039
Epoch 6/10
179/179 [==============================] - 175s 974ms/step - loss: 0.0887 - accuracy: 0.9684 - val_loss: 0.0751 - val_accuracy: 0.9705
Epoch 7/10
179/179 [==============================] - 170s 951ms/step - loss: 0.0426 - accuracy: 0.9837 - val_loss: 0.3300 -

In [5]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

45/45 [==============================] - 12s 259ms/step - loss: 0.0528 - accuracy: 0.9853
Test accuracy: 0.9852734804153442


In [6]:
import matplotlib.pyplot as plt
import cv2

### Saving the Model

In [7]:

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model to a file
with open('potato_disease.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /var/folders/wc/4vhbxzyn749cv5d6p5s6nw100000gp/T/tmpyofu4wyg/assets


INFO:tensorflow:Assets written to: /var/folders/wc/4vhbxzyn749cv5d6p5s6nw100000gp/T/tmpyofu4wyg/assets
2024-04-19 15:46:49.501106: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-04-19 15:46:49.501235: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-04-19 15:46:49.504205: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/wc/4vhbxzyn749cv5d6p5s6nw100000gp/T/tmpyofu4wyg
2024-04-19 15:46:49.505019: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-04-19 15:46:49.505024: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/wc/4vhbxzyn749cv5d6p5s6nw100000gp/T/tmpyofu4wyg
2024-04-19 15:46:49.508145: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-04-19 15:46:49.508805: I tensorflow/cc/saved_model/load

In [8]:
model.save("potato_model.h5")

/Users/student/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Predictions

In [10]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the image you want to predict
img_path = 'potato_early.JPG'  # Provide the path to your image
img = image.load_img(img_path, target_size=(250, 250))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.  # Rescale pixel values to [0, 1]

# Make prediction
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)

# Map predicted class index to class label
class_labels = train_generator.class_indices
predicted_label = [k for k, v in class_labels.items() if v == predicted_class][0]

print("Predicted class:", predicted_label)
for i in range(0,3):
    prediction[0][i] = np.round((prediction[0][i]),2)
print("Prediction probabilities:", prediction)


1/1 [==============================] - 0s 16ms/step
Predicted class: Potato___Early_blight
Prediction probabilities: [[1. 0. 0.]]


In [11]:
# Load the image you want to predict
img_path = 'potato_healthy.JPG'  # Provide the path to your image
img = image.load_img(img_path, target_size=(250, 250))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.  # Rescale pixel values to [0, 1]

# Make prediction
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)

# Map predicted class index to class label
class_labels = train_generator.class_indices
predicted_label = [k for k, v in class_labels.items() if v == predicted_class][0]

print("Predicted class:", predicted_label)
for i in range(0,3):
    prediction[0][i] = np.round((prediction[0][i]),2)
print("Prediction probabilities:", prediction)


1/1 [==============================] - 0s 15ms/step
Predicted class: Potato___healthy
Prediction probabilities: [[0. 0. 1.]]


In [12]:
# Load the image you want to predict
img_path = 'potato_lateb.JPG'  # Provide the path to your image
img = image.load_img(img_path, target_size=(250, 250))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.  # Rescale pixel values to [0, 1]

# Make prediction
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)

# Map predicted class index to class label
class_labels = train_generator.class_indices
predicted_label = [k for k, v in class_labels.items() if v == predicted_class][0]

print("Predicted class:", predicted_label)
for i in range(0,3):
    prediction[0][i] = np.round((prediction[0][i]),2)
print("Prediction probabilities:", prediction)


1/1 [==============================] - 0s 15ms/step
Predicted class: Potato___Late_blight
Prediction probabilities: [[0. 1. 0.]]
